<a href="https://colab.research.google.com/github/tayfununal/Normalizing-Flows/blob/main/ode_solver_using_hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Solving Differential Equation**

## **Hamiltonian Mechanics**

\begin{align}
        H(q,p) = \frac {q^2}{2} + \frac {p^2}{2}
\end{align}


\begin{align}
     q(t) = sin(t), p(t) = cos(t) \implies \dot{q} = \frac{dq(t)}{dt} = cos(t) = p \text{ ve } \dot{p} = \frac{dp(t)}{dt} = -sin(t) = -q
\end{align}


\begin{align}
        \frac{ \partial H(q(t), p(t))}{ \partial t} = \begin{pmatrix}
         \dot{q} \\
         \dot{p} 
        \end{pmatrix} = \begin{pmatrix}
         p \\
         -q 
        \end{pmatrix}
    \end{align}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp

plt.style.use('seaborn-poster')

%matplotlib inline

def F(t,s):
  return [np.cos(t), -np.sin(t)]

t_eval = np.arange(0, 16*np.pi, 0.01)
sol = solve_ivp(F, [0, 16*np.pi], [0., 1.], t_eval=t_eval, dense_output=True)


plt.figure(figsize = (12, 6))

plt.subplot(221)
plt.plot(sol.t, sol.y[0])
plt.xlabel('t')
plt.ylabel('H[0] = sin(t)')

plt.subplot(222)
plt.plot(sol.t, sol.y[0] - np.sin(sol.t))
plt.xlabel('t')
plt.ylabel('H[0] - sin(t)')

plt.subplot(223)
plt.plot(sol.t, sol.y[1])
plt.xlabel('t')
plt.ylabel('H[1] = cos(t)')

plt.subplot(224)
plt.plot(sol.t, sol.y[1] - np.cos(sol.t))
plt.xlabel('t')
plt.ylabel('H[1] - cos(t)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(sol.y.T[:, 0], sol.y.T[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
y = sol.y.T

In [ ]:
y.shape # (q,p)

In [ ]:
t = sol.t

In [ ]:
from tensorflow.python.ops.gen_math_ops import sin
model = Sequential([
    Dense(64, activation= 'tanh',input_shape=(1,)),
    Dense(16, activation= 'tanh'),
    Dense(64, activation= 'tanh'),
    Dense(32, activation= 'tanh'),
    Dense(16, activation='tanh'),

    Dense(2,)
])

In [ ]:
model.summary()

In [ ]:
import keras.backend as K
def custom_loss(y_true, y_pred):

    #y_true = K.print_tensor(y_true, message='y_true = ')
    #y_pred = K.print_tensor(y_pred, message='y_pred = ')
    
    # calculate loss, using y_pred
    #y_true = np.array([[1/2],[0]]).reshape((1,2))      # H(q,p) = q^2/2 + p^2/2   , q(t=0)=cos(0)=1, p(t=0)=sin(0)=0 
    #y_pred = tf.matmul(tf.cast(y_pred, tf.float64), tf.convert_to_tensor(np.array([[0., 1.],[-1., 0.]], dtype='float64')))

    y_true = tf.reduce_sum((y_true**2)/2)
    y_pred = tf.reduce_sum((y_pred**2)/2)

    loss = (y_true - y_pred)**2

    return loss

In [ ]:
model.compile(optimizer='adam', loss=custom_loss)

In [ ]:
model.fit(x=t, y= y, epochs=200)

In [ ]:
t_deneme = np.arange(0, 2*np.pi, 0.02).reshape((315,1))
pred = model.predict(t_deneme)

In [ ]:
plt.figure(figsize = (12, 8))
plt.plot(pred[:, 0], pred[:, 1])
plt.xlabel('q')
plt.ylabel('p')
plt.show()